[View in Colaboratory](https://colab.research.google.com/github/msnaru/NaroRecommender/blob/master/TestScrapingNaro.ipynb)

In [0]:
# モジュールのインストール

!pip install BeautifulSoup4
!pip install lxml

    100% |████████████████████████████████| 5.8MB 4.9MB/s 


In [0]:
# モジュールのimport

import pandas as pd
from time import sleep
import urllib.request
from bs4 import BeautifulSoup

In [0]:
# 関数定義


# 感想の最大ページ数抽出
def MaxPage(url):
  # 実際の最大ページ数を超えるページにアクセスすると、実際の最大ページにアクセスされる
  url += "?p=10000"
  
  try:
    xml = urllib.request.urlopen(url)
    soup = BeautifulSoup(xml, "lxml")
    naviall = soup.find_all("div", class_ = 'naviall')
    if len(naviall) > 0:
      strNaviall = repr(naviall[1].text.split(" ")[1]).split("\\xa")
      nMaxPpage = int(strNaviall[len(strNaviall) - 1][:-1])
      return nMaxPpage
    else:
      return 1
  
  except:
    print("You cannot get a number of page.")
        
      
# 感想スクレイピング処理
def scraping(url, lenURL, i):
    xml = urllib.request.urlopen(url)
    soup = BeautifulSoup(xml, "lxml")
    comments = soup.find_all("div", class_='comment')
    
    df_add = pd.DataFrame(columns=["impression"])
    for comment in comments:
        comment = comment.text
        df_add = df_add.append(pd.DataFrame([[comment]], columns=["impression"]))[df_add.columns.tolist()]
    

    next_url = url[:lenURL] + "?p=" + str(i+2)
    
    return df_add,next_url
  
  
# 感想抽出処理
def Extract_imp(url):

  lenURL = len(url)

  # 取得する感想のページ数。最大ページ数を制限
  nMaxPage = MaxPage(url)
  if nMaxPage > 100:
    nMaxPage = 100

  df = pd.DataFrame(columns=["impression"])

  # scraping(url) を呼び出してスクレイピングを実行
  for i in range(nMaxPage):
    df_add, url= scraping(url, lenURL, i)
    df = df.append(df_add,ignore_index=True)
    sleep(1)

  # スクレイピングしたデータに重複データがあることを確認
  print("データの個数："+str(len(df)))

  #データに重複があるものは全て削除
  df_cleansing = df.drop_duplicates(keep=False)
  print("データの個数："+str(len(df_cleansing)))
  
  return [df, df_cleansing]


# 自分の評価済み小説のタイトルと評価のリストを取得
def Evaluated_title():

  url = "https://mypage.syosetu.com/mypagenovelhyoka/list/userid/63023/"
  xml = urllib.request.urlopen(url)
  soup = BeautifulSoup(xml, "lxml")
  titles = soup.find_all("li", class_ ='title')
  hyokas = soup.find_all("p", class_ ='hyouka')


  title_list = []

  for title in titles:
      title = repr(title.text)
      title_list.append((title[:-4]).split("(")[1])

  pd_title = pd.DataFrame({'title': title_list})
  

  i = 0
  hyoka_pair_list = []
  hyoka_pair = []

  for hyoka in hyokas:
    hyoka = repr(hyoka.text)
    i += 1
    if i % 2 == 0:
      hyoka_pair.append(int(hyoka[-4:][:1]))
      hyoka_pair_list.append(hyoka_pair)
      hyoka_pair = []
    else:
      hyoka_pair.append(int(hyoka[-4:][:1]))

      
  pd_hyoka = pd.DataFrame(hyoka_pair_list)
  pd_hyoka = pd_hyoka.rename(columns={0: 'ev_story', 1: 'ev_sentence'})
  
  pd_Evaluated = pd.concat([pd_title, pd_hyoka], axis=1)
  return pd_Evaluated


# 任意のタイトルのトップリンクと感想リンクの抽出
def links(title):

  link_title = "https://ncode.syosetu.com/" + title + "/"


  xml = urllib.request.urlopen(link_title)
  soup = BeautifulSoup(xml, "lxml")
  comments = soup.find_all("div", id='novel_header')

  l = str(comments).splitlines()

  l_in = [s for s in l if 'https://novelcom.syosetu.com/impression/list/ncode/' in s]
  link_imp = l_in[0].split('"')[1]


  return [link_title, link_imp]


def add_links(df):
  try:
    pd_links = pd.DataFrame([links(title) for title in df.loc[:,"title"].tolist()])
    pd_links = pd_links.rename(columns={0: 'link_title', 1: 'link_impression'})

    return pd.concat([df, pd_links], axis = 1)
  
  except:
    print("入力DataFrameが正しくありません")

In [0]:
df = Evaluated_title()
df2 = add_links(df)

impressions = [Extract_imp(url) for url in df2.loc[:, "link_impression"].tolist()]

for i in range(10):
  #CSVでデータを保存
  impressions[i][0].to_csv("impression%d.csv"% i, index=False, encoding="UTF-8")
  impressions[i][1].to_csv("impression%d_cleansed.csv"% i, index=False, encoding="UTF-8")


データの個数：1549
データの個数：1503
データの個数：219
データの個数：148
データの個数：1078
データの個数：1038
データの個数：337
データの個数：318
データの個数：22
データの個数：22
データの個数：6
データの個数：6
データの個数：1325
データの個数：1311
データの個数：2214
データの個数：2214
データの個数：492
データの個数：484
データの個数：1111
データの個数：1111
